In [1]:
import cv2
import numpy as np

# 예제 8.1 cv2.preCornerDetect() 코너점 검출

In [2]:
def findLocalMaxima(src):
    kernel = cv2.getStructuringElement(shape = cv2.MORPH_RECT, ksize = (11,11)) # 1로 채워진 11*11 크기의 배열
    dilate = cv2.dilate(src, kernel) # 팽창: 전경이 되는 이미지의 경계 부분을 팽창 -> 전경 이미지가 굵어지는 효과
    localMax = (src == dilate) # 픽셀값의 변화가 없는 곳(배경이나 전경 이미지)은 1, 변화가 생긴 곳(전경 이미지 굻어지는 부분)은 0
    # print(localMax)
    erode = cv2.erode(src, kernel) # 침식: 전경이 되는 이미지의 경계 부분을 침식시켜 배경 이미지로 전환 -> 전경 이미지가 가늘어지는 효과
    localMax2 = src > erode
    # print(localMax2)
    localMax &= localMax2
    points = np.argwhere(localMax == True)
    points[:,[0,1]] = points[:,[1,0]]
    return points

<details>
<summary>erosion 이미지</summary>
<div markdown="1">
<img src="https://user-images.githubusercontent.com/84698896/213682801-d5c866e2-2241-4387-be50-8f6f843cd443.png"  width="20%" height="20%"/>
</div>
</details>

### <span style='background-color:#fff5b1'>cv2.preCornerDetect(src, ksize)</span>
src에서 코너점 검출을 위한 특징맵 dst를 sobel 미분 연산자를 이용하여 계산

In [3]:
src = np.zeros(shape = (512,512,3), dtype = np.uint8) + 255
cv2.rectangle(src, (100,100), (300,300), (0,0,0), -1)
cv2.rectangle(src, (200,200), (400,400), (0,0,0), -1)

gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.preCornerDetect(gray, ksize=3)
ret, res2 = cv2.threshold(np.abs(res), 0.1, 0, cv2.THRESH_TOZERO)
corners = findLocalMaxima(res2)
corners.shape

(8, 2)

In [4]:
dst = src.copy()
for x, y in corners:
    cv2.circle(dst, (x,y), 5, (0,0,255), 2)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

# 8.2 코너점 검출: cornerEigenValsAndVecs()
### <span style='background-color:#fff5b1'>cv2.cornerDigenValsAndVecs(src, blockSize, ksize)</span>
src에서 각 화소의 고유값과 고유 벡터를 6-채널 dst에 계산 \
이웃: blockSize * blockSize

In [5]:
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.cornerEigenValsAndVecs(gray, blockSize = 5, ksize = 3)
eigen = cv2.split(res)
res.shape

(512, 512, 6)

In [6]:
T = 0.2
ret, edge = cv2.threshold(eigen[0], T, 255, cv2.THRESH_BINARY)
edge = edge.astype(np.uint8)

In [7]:
corners = np.argwhere(eigen[1] > T)
corners[:,[0,1]] = corners[:,[1,0]]
len(corners)

8

In [8]:
dst = src.copy()
for x, y in corners:
    cv2.circle(dst, (x,y), 5, (0,0,255), 2)

cv2.imshow('edge', edge)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

# 예제 8.3 코너점 검출 1: cv2.cornerMinEigenVal()
### <span style='background-color:#fff5b1'>cv2.cornerMinEigenVal(src, blockSize = 5)</span>
src에서 각 화소의 최소 고유값을 dst로 반환

In [9]:
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
eigen = cv2.cornerMinEigenVal(gray, blockSize=5)
eigen.shape

(512, 512)

In [10]:
T = 0.2
corners = np.argwhere(eigen > T)
corners[:,[0,1]] = corners[:,[1,0]]
len(corners)

8

In [11]:
dst = src.copy()
for x, y in corners:
    cv2.circle(dst, (x,y), 3, (0,0,255), 2)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()